# Vesuvius Challenge Surface Detection - Data Exploration

This notebook explores the 3D CT volume data for surface detection.

## Goals:
1. Load and visualize 3D volumes
2. Understand data structure and characteristics
3. Visualize augmentations
4. Test baseline Sobel detector
5. Explore ground truth labels

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch

# Import local modules
from src.data.preprocessing import VolumeLoader, PatchExtractor
from src.data.augmentations import VolumeAugmentationPipeline, visualize_augmentations
from src.models.sobel_baseline import SobelSurfaceDetector, visualize_surface_detection

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

## 1. Load 3D Volume Data

In [ ]:
# Configure data path
data_root = Path("../data/raw")
fragment_id = "fragment_1"  # Change this to your fragment ID

# Check if data exists
if not (data_root / fragment_id).exists():
    print(f"⚠️ Data not found at {data_root / fragment_id}")
    print("Please download fragment data first.")
    print("\nYou can download data from:")
    print("- https://dl.ash2txt.org/")
    print("- Kaggle Vesuvius Challenge datasets")
else:
    print(f"✓ Data found at {data_root / fragment_id}")

In [ ]:
# Load volume
loader = VolumeLoader(
    data_root=data_root,
    fragment_id=fragment_id,
    num_slices=65,
    normalize=True
)

volume = loader.load_volume()
mask = loader.load_mask()
labels = loader.load_labels()

print(f"Volume shape: {volume.shape}")
print(f"Volume dtype: {volume.dtype}")
print(f"Value range: [{volume.min():.3f}, {volume.max():.3f}]")

if mask is not None:
    print(f"\nMask shape: {mask.shape}")
    print(f"Valid region: {mask.sum() / mask.size * 100:.2f}%")

if labels is not None:
    print(f"\nLabels shape: {labels.shape}")
    print(f"Surface coverage: {labels.sum() / labels.size * 100:.2f}%")

## 2. Visualize 3D Volume

In [ ]:
# Visualize different slices
slice_indices = [0, 16, 32, 48, 64]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i, idx in enumerate(slice_indices):
    axes[i].imshow(volume[idx], cmap='gray')
    axes[i].set_title(f'Slice {idx}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Maximum intensity projection
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Z-axis projection (looking down)
axes[0].imshow(volume.max(axis=0), cmap='gray')
axes[0].set_title('Max Projection (Z-axis)')
axes[0].axis('off')

# Y-axis projection (looking from side)
axes[1].imshow(volume.max(axis=1), cmap='gray')
axes[1].set_title('Max Projection (Y-axis)')
axes[1].axis('off')

# X-axis projection (looking from side)
axes[2].imshow(volume.max(axis=2), cmap='gray')
axes[2].set_title('Max Projection (X-axis)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 3. Visualize Mask and Labels

In [ ]:
if mask is not None and labels is not None:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Middle slice
    axes[0].imshow(volume[32], cmap='gray')
    axes[0].set_title('Volume (Slice 32)')
    axes[0].axis('off')
    
    # Mask
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title('Valid Region Mask')
    axes[1].axis('off')
    
    # Labels
    axes[2].imshow(labels, cmap='hot')
    axes[2].set_title('Surface Labels (Ground Truth)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Overlay
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(volume[32], cmap='gray')
    ax.imshow(labels, cmap='hot', alpha=0.3)
    ax.set_title('Volume with Surface Overlay')
    ax.axis('off')
    plt.show()

## 4. Test Augmentations

In [ ]:
# Extract a small patch for testing
test_volume = volume[:, :256, :256]
test_mask = mask[:256, :256] if mask is not None else None

# Visualize augmentations
if test_mask is not None:
    visualize_augmentations(test_volume, test_mask, n_examples=3)
else:
    print("No mask available for augmentation visualization")

## 5. Test Baseline Sobel Detector

In [ ]:
# Initialize detector
detector = SobelSurfaceDetector(
    gradient_threshold=0.1,
    use_second_derivative=True,
    morphology_radius=2,
    min_component_size=100
)

# Detect surface on small patch
test_volume_small = volume[:, :512, :512]

binary_mask, debug_info = detector.detect_surface(
    test_volume_small,
    umbilicus_center=None,  # Set if you know scroll center
    smooth_volume=True,
    sigma=1.0
)

print(f"\nDetected surface coverage: {debug_info['surface_coverage']*100:.2f}%")

In [ ]:
# Visualize results
visualize_surface_detection(
    test_volume_small,
    binary_mask,
    debug_info,
    slice_idx=32
)

## 6. Extract Training Patches

In [ ]:
# Initialize patch extractor
extractor = PatchExtractor(
    patch_size=256,
    stride=128,
    balanced_sampling=True,
    surface_ratio=0.5
)

# Extract patches
if labels is not None:
    vol_patches, label_patches, coords = extractor.extract_patches(
        volume, mask, labels
    )
    
    print(f"Extracted {len(vol_patches)} patches")
    
    if vol_patches:
        print(f"Volume patch shape: {vol_patches[0].shape}")
        print(f"Label patch shape: {label_patches[0].shape}")
        
        # Count surface vs non-surface patches
        surface_patches = sum(1 for lp in label_patches if lp.sum() > 0)
        print(f"\nSurface patches: {surface_patches}")
        print(f"Non-surface patches: {len(label_patches) - surface_patches}")
else:
    print("No labels available for patch extraction")

In [ ]:
# Visualize some patches
if len(vol_patches) > 0:
    n_examples = min(4, len(vol_patches))
    
    fig, axes = plt.subplots(n_examples, 3, figsize=(12, 4*n_examples))
    
    for i in range(n_examples):
        # Volume patch (middle slice)
        axes[i, 0].imshow(vol_patches[i][32], cmap='gray')
        axes[i, 0].set_title(f'Patch {i} - Volume (Slice 32)')
        axes[i, 0].axis('off')
        
        # Max projection
        axes[i, 1].imshow(vol_patches[i].max(axis=0), cmap='gray')
        axes[i, 1].set_title(f'Patch {i} - Max Projection')
        axes[i, 1].axis('off')
        
        # Label
        axes[i, 2].imshow(label_patches[i], cmap='hot')
        axes[i, 2].set_title(f'Patch {i} - Label')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

## 7. Data Statistics

In [ ]:
# Volume statistics
print("Volume Statistics:")
print("="*50)
print(f"Shape: {volume.shape}")
print(f"Min value: {volume.min():.3f}")
print(f"Max value: {volume.max():.3f}")
print(f"Mean value: {volume.mean():.3f}")
print(f"Std value: {volume.std():.3f}")
print(f"Memory size: {volume.nbytes / 1024**2:.2f} MB")

# Histogram
plt.figure(figsize=(10, 4))
plt.hist(volume.flatten(), bins=100, alpha=0.7)
plt.xlabel('Intensity Value')
plt.ylabel('Frequency')
plt.title('Volume Intensity Distribution')
plt.show()

## 8. Next Steps

After exploring the data:
1. Download more fragment data if needed
2. Train baseline Sobel detector on full volume
3. Train 3D U-Net model (see training notebooks)
4. Experiment with different augmentation strategies
5. Evaluate on validation data